<a href="https://colab.research.google.com/github/1kaiser/Snow-cover-area-estimation/blob/main/tiff2array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/OUT/data/data20230320 /content/

^C


In [3]:
import os
image_dir = r'/content/data20230320/'

#############################################################################>> files are of their original sizes are kept in a folder with their respective extensions as bands b1, b2, b3 , b4 , b5 ....  and kept as to be listed as image list in a variable.
prefix = ""
end = ["B11", "B12", "B02", "B03", "B04", "B05", "B06", "B07", "B8A", "B08"]
DayOY = "_doy\[0-9]+_aid0001"
fileExt = r'.tif'
expression_b1 = prefix+end[0]
expression_b2 = prefix+end[1]
expression_b3 = prefix+end[2]
expression_b4 = prefix+end[3]
expression_b5 = prefix+end[4]
expression_b6 = prefix+end[5]
expression_b7 = prefix+end[6]
expression_b8 = prefix+end[7]
expression_b9 = prefix+end[8]
expression_b10 = prefix+end[9]

imgs_list_b1 = [f for f in os.listdir(image_dir) if f.__contains__(expression_b1)]

imgs_list_b1.sort(reverse=True)    
print(imgs_list_b1)

['download.B11.tif']


In [4]:
path = os.path.join(image_dir, imgs_list_b1[0])
path_list = [path.replace(expression_b1, expression_b1),
             path.replace(expression_b1, expression_b2),
             path.replace(expression_b1, expression_b3),
             path.replace(expression_b1, expression_b4),
             path.replace(expression_b1, expression_b5),
             path.replace(expression_b1, expression_b6),
             path.replace(expression_b1, expression_b7),
             path.replace(expression_b1, expression_b8),
             path.replace(expression_b1, expression_b9),
             path.replace(expression_b1, expression_b10)]

path_list

['/content/data20230320/download.B11.tif',
 '/content/data20230320/download.B12.tif',
 '/content/data20230320/download.B02.tif',
 '/content/data20230320/download.B03.tif',
 '/content/data20230320/download.B04.tif',
 '/content/data20230320/download.B05.tif',
 '/content/data20230320/download.B06.tif',
 '/content/data20230320/download.B07.tif',
 '/content/data20230320/download.B8A.tif',
 '/content/data20230320/download.B8x.tif']

In [ ]:

path_list[0][-7:-4]

'B11'

In [5]:
#this function is for getting the four corners of all the square boxes possible out of the complete image size >>>
def get_slice_bboxes( image_height: int, image_width: int, slice_height: int = 2000 , slice_width: int = 2000, overlap_height_ratio: float = 0.0, overlap_width_ratio: float = 0.0 ):
  slice_bboxes = []
  y_max = y_min = 0

  if slice_height and slice_width:
      y_overlap = int(overlap_height_ratio * slice_height)
      x_overlap = int(overlap_width_ratio * slice_width)
  else:
      raise ValueError("Compute type is not auto and slice width and height are not provided.")

  while y_max < image_height:
      x_min = x_max = 0
      y_max = y_min + slice_height
      while x_max < image_width:
          x_max = x_min + slice_width
          if y_max > image_height or x_max > image_width:
              xmax = min(image_width, x_max)
              ymax = min(image_height, y_max)
              xmin = max(0, xmax - slice_width)
              ymin = max(0, ymax - slice_height)
              slice_bboxes.append([xmin, ymin, xmax, ymax])
          else:
              slice_bboxes.append([x_min, y_min, x_max, y_max])
          x_min = x_max - x_overlap
      y_min = y_max - y_overlap
  return slice_bboxes


In [ ]:
#@title Default title text { vertical-output: true }
# functions for clipping a single raster to multiple rasters of various shapes
from osgeo import gdal

save_directory = "/content/drive/MyDrive/OUT/data/data20230320/inputs/"
!mkdir -p {save_directory} 




band 0

In [ ]:
here_image = path_list[0]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 1

In [ ]:
here_image = path_list[1]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 2

In [ ]:
here_image = path_list[2]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 3

In [ ]:
here_image = path_list[3]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 4

In [ ]:
here_image = path_list[4]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 5

In [ ]:
here_image = path_list[5]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 6

In [ ]:
here_image = path_list[6]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 7

In [ ]:
here_image = path_list[7]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 8

In [ ]:
here_image = path_list[8]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

band 9

In [ ]:
here_image = path_list[9]
print(here_image)

vvv = here_image[-7:-4]
ds = gdal.Open(here_image)
box_list = get_slice_bboxes(ds.RasterYSize , ds.RasterXSize)

count = 0

for box in box_list:
  print(box, type(box_list))
  count = count + 1
  gdal.Translate(str(save_directory) + str(vvv) + "." + str(count) + '.tif', here_image ,srcWin = box )
!runtime.unassign()

## test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

def get_gain_band(input_file):
    """get GAIN_BAND from meta file (*.tif.txt)"""
     # define file name of *.tif.txt
    ifile_txt = re.sub(r'.tif', '.tif.txt', input_file)
    ld = open(ifile_txt)
    lines = ld.readlines()
    ld.close()
    
    gain_band = []
    for line in lines:
        if line.find("GAIN_BAND") >= 0:
             gain_band.append(float((re.split(' ', line)[1]).strip()))
    return gain_band

def tif2array(input_file, calc_gain=True):
    """
    read GeoTiff and convert to numpy.ndarray.
    Inputs:
        input_file (str) : the name of input GeoTiff file.
        calc_gain (bool) : wheter calc GAIN to DN  or not (defaul:True).
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=float)
    
    if calc_gain == True:
        # get gain
        gain = get_gain_band(input_file)
    
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if calc_gain == True:
            # calc gain value for each bands
            image[:, :, b] = band.ReadAsArray() * gain[b]
        else:
            image[:, :, b] = band.ReadAsArray()
    return image, dataset

setting up for tif2array

In [ ]:
import os
image_dir = r'/content/drive/MyDrive/OUT/data/data20230320/inputs/'

#############################################################################>> files are of their original sizes are kept in a folder with their respective extensions as bands b1, b2, b3 , b4 , b5 ....  and kept as to be listed as image list in a variable.
prefix = ""
end = ["B11", "B12", "B02", "B03", "B04", "B05", "B06", "B07", "B8A", "B08"]
DayOY = "_doy\[0-9]+_aid0001"
fileExt = r'.tif'
expression_b1 = prefix+end[0]
expression_b2 = prefix+end[1]
expression_b3 = prefix+end[2]
expression_b4 = prefix+end[3]
expression_b5 = prefix+end[4]
expression_b6 = prefix+end[5]
expression_b7 = prefix+end[6]
expression_b8 = prefix+end[7]
expression_b9 = prefix+end[8]
expression_b10 = prefix+end[9]

imgs_list_b1 = [f for f in os.listdir(image_dir) if f.__contains__(expression_b1)]

imgs_list_b1.sort(reverse=True)    
print(imgs_list_b1)



In [ ]:
#extraction of band values from clips to DATAFRAME for to be processing inside the model

from osgeo import gdal
import pandas as pd


######################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$<<< tensorflow model
#Copy and execute the following code in a new Google Colab notebook launch to run the model.
!python -m pip install tensorflow tensorflow_decision_forests -U -qq

# Transfer the model from Google Drive to Colab
from google.colab import drive
drive.mount("/content/gdrive")
!cp "/content/gdrive/My Drive/simple_ml_for_sheets/Ice height from bands" ydf_model
  
# Prepare and load the model with TensorFlow
import tensorflow as tf
import tensorflow_decision_forests as tfdf

tfdf.keras.yggdrasil_model_to_keras_model("ydf_model", "tfdf_model")
model = tf.keras.models.load_model("tfdf_model")
######################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$<<< tensorflow model


In [ ]:


for image_no in range(0, len(imgs_list_b1)):
  path = os.path.join(image_dir, imgs_list_b1[image_no])
  path_list = [path.replace(expression_b1, expression_b1),
              path.replace(expression_b1, expression_b2),
              path.replace(expression_b1, expression_b3),
              path.replace(expression_b1, expression_b4),
              path.replace(expression_b1, expression_b5),
              path.replace(expression_b1, expression_b6),
              path.replace(expression_b1, expression_b7),
              path.replace(expression_b1, expression_b8),
              path.replace(expression_b1, expression_b9),
              path.replace(expression_b1, expression_b10)]

  lon , lat = [] , []  
  v1 =[]

  for i in path_list:
    pathout = temp_dir+str(i)
    print(pathout)
    
    if(path_list.index(i)==0):
      v1 = tif2array(pathout, 0)[0].reshape(-1,1)
      # Open tif file
      ds = gdal.Open(pathout)
      # GDAL affine transform parameters, According to gdal documentation xoff/yoff are image left corner, a/e are pixel wight/height and b/d is rotation and is zero if image is north up. 
      xoff, a, b, yoff, d, e = ds.GetGeoTransform()
      print(xoff, a, b, yoff, d, e)
      def pixel2coord(x, y):
        # ""Returns global coordinates from pixel x, y coords""
        xp = a * x + b * y + xoff
        yp = d * x + e * y + yoff
        return xp, yp

      # get columns and rows of your image from gdalinfo
      rows = ds.RasterYSize 
      colms = ds.RasterXSize 
      for row in  range(0,rows):
        for col in  range(0,colms): 
          lat.append(pixel2coord(col,row)[0])
          lon.append(pixel2coord(col,row)[1])
    else:
      v1 = np.append(v1, tif2array(pathout, 0)[0].reshape(-1,1), axis = 1)
  # agging latitude "lat" & longitde "lon" column at the end
  v1 = np.column_stack((v1, np.column_stack((lat, lon))))


  arr = v1
  # convert array into dataframe
  DF = pd.DataFrame(arr)
  # DF.columns = d
    # print(DF)
  height =[]
  for values in range(0, len(DF[0])):
    examples = {
    "b11" : [np.array(DF[0][values])],
    "b12" : [np.array(DF[1][values])],
    "b2" : [np.array(DF[2][values])],
    "b3" : [np.array(DF[3][values])],
    "b4" : [np.array(DF[4][values])],
    "b5" : [np.array(DF[5][values])],
    "b6" : [np.array(DF[6][values])],
    "b7" : [np.array(DF[7][values])],
    "b8a" : [np.array(DF[8][values])],
    "b8" : [np.array(DF[9][values])],
    "lat" : [np.array(DF[10][values])],
    "lon" : [np.array(DF[11][values])],
    }
######################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$<<< tensorflow model

    a = np.array(model.predict_step(examples))
######################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$<<< tensorflow model

    height.append(a[0][0])
  print(np.asarray(height))
  print(height)
  height = np.column_stack((np.column_stack((lat, lon)), height))
  # saving the predicted heght data to panda dataframe
  DHeight = pd.DataFrame(height)
  # DF.columns = d
  DHeight
  # save the dataframe as a csv file
  DHeight.to_csv("height_" + str(image_no) + ".csv")
